In [1]:
import  tensorflow as tf
from    tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

In [2]:
def preprocess(x, y):

    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)

    return x,y

In [3]:
batchsz = 128
(x, y), (x_val, y_val) = datasets.mnist.load_data()
print('datasets:', x.shape, y.shape, x.min(), x.max())


datasets: (60000, 28, 28) (60000,) 0 255


In [4]:
db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(60000).batch(batchsz).repeat(10)

ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz) 

In [15]:
network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(128, activation='relu'),
                     layers.Dense(64, activation='relu'),
                     layers.Dense(32, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(None, 28*28))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [6]:
optimizer = optimizers.Adam(lr=0.01)

In [7]:
acc_meter = metrics.Accuracy()
loss_meter = metrics.Mean()

In [14]:
for step, (x,y) in enumerate(db):

    with tf.GradientTape() as tape:
        # [b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # [b, 784] => [b, 10]
        out = network(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10) 
        # [b]
        loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot, out, from_logits=True))

        loss_meter.update_state(loss)

 

    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))


    if step % 100 == 0:

        print(step, 'loss:', loss_meter.result().numpy()) 
        loss_meter.reset_states()


    # evaluate
    if step % 500 == 0:
        total, total_correct = 0., 0
        acc_meter.reset_states()

        for step, (x, y) in enumerate(ds_val): 
            # [b, 28, 28] => [b, 784]
            x = tf.reshape(x, (-1, 28*28))
            # [b, 784] => [b, 10]
            out = network(x) 


            # [b, 10] => [b] 
            pred = tf.argmax(out, axis=1) 
            pred = tf.cast(pred, dtype=tf.int32)
            # bool type 
            correct = tf.equal(pred, y)
            # bool tensor => int tensor => numpy
            total_correct += tf.reduce_sum(tf.cast(correct, dtype=tf.int32)).numpy()
            total += x.shape[0]

            acc_meter.update_state(y, pred)


        print(step, 'Evaluate Acc:', total_correct/total, acc_meter.result().numpy())

0 loss: 0.064001225
78 Evaluate Acc: 0.973 0.973
100 loss: 0.0464973
200 loss: 0.050142054
300 loss: 0.07124271
400 loss: 0.06828959
500 loss: 0.083423354
78 Evaluate Acc: 0.976 0.976
600 loss: 0.043216944
700 loss: 0.056067467
800 loss: 0.06528304
900 loss: 0.06486122
1000 loss: 0.07151028
78 Evaluate Acc: 0.9742 0.9742
1100 loss: 0.05613645
1200 loss: 0.06587656
1300 loss: 0.06232592
1400 loss: 0.06376213
1500 loss: 0.053104922
78 Evaluate Acc: 0.9699 0.9699
1600 loss: 0.065917365
1700 loss: 0.07090145
1800 loss: 0.06511085
1900 loss: 0.054743595
2000 loss: 0.055892676
78 Evaluate Acc: 0.975 0.975
2100 loss: 0.05515947
2200 loss: 0.059773516
2300 loss: 0.059633184
2400 loss: 0.046936747
2500 loss: 0.05151545
78 Evaluate Acc: 0.9701 0.9701
2600 loss: 0.06942699
2700 loss: 0.054700635
2800 loss: 0.05086112
2900 loss: 0.043505613
3000 loss: 0.03180887
78 Evaluate Acc: 0.9805 0.9805
3100 loss: 0.050394915
3200 loss: 0.049254056
3300 loss: 0.05291812
3400 loss: 0.04579306
3500 loss: 0.054

In [12]:
summary_writer = tf.summary.create_file_writer("logs/data")


In [13]:
with summary_writer.as_default():
    tf.summary.scalar('train-loss',float(loss),step=step)